# Imports

In [2]:
import pandas as pd 
import numpy as np
import pickle

from PIL import Image

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# Reading csv and spliting data

In [3]:
df_train = pd.read_csv('data\mnist_train.csv')
df_train

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df_train = df_train.reindex(np.random.permutation(df_train.index))

In [5]:
# df_train = df_train.drop(df_train.index[5000:]) # Don't run more than once

In [6]:
df_train.shape

(60000, 785)

In [7]:
X_train = df_train.drop(columns = ['label'])
X_train

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
28910,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47506,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50373,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57565,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51488,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16805,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38691,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
y_train = df_train['label']
y_train

28910    0
49157    3
47506    8
50373    9
57565    6
        ..
24049    7
51488    8
16805    1
38691    0
2924     9
Name: label, Length: 60000, dtype: int64

In [9]:
df_test = pd.read_csv('data\mnist_test.csv')
df_test

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df_test = df_test.reindex(np.random.permutation(df_test.index))

In [16]:
# df_test = df_test.drop(df_test.index[999:9999]) # Don't run more than once

In [11]:
df_test.shape

(10000, 785)

In [12]:
X_test = df_test.drop(columns = ['label'])
X_test

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
6300,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8488,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6112,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1263,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5778,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9653,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9192,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
534,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
y_test = df_test['label']
y_test

6300    4
6498    6
8488    1
6112    9
1263    4
       ..
5778    0
9653    1
9192    3
534     6
4642    0
Name: label, Length: 10000, dtype: int64

# Random Forest Classifier with Hyperparameter Optimization

In [14]:
rf_classifier = RandomForestClassifier()

In [15]:
param_space = {'n_estimators' : [10,50,80,90,100,1000], 
               'max_depth' : [10,70,80,90,100,1000], 
               'min_samples_split' : [10,50,60,70,80,90,1000], 
               'min_samples_leaf' : [1,10,60,80,90,100,1000], 
               'max_features' : [2,4,6,8,10,100,1000]}

Already used cell below to find best params

In [16]:
rf_classifier_cv = RandomizedSearchCV(rf_classifier, param_space, n_iter=10, scoring = 'neg_root_mean_squared_error', cv = 5 )
search = rf_classifier_cv.fit(X_train, y_train)

In [17]:
search.best_params_

{'n_estimators': 90,
 'min_samples_split': 80,
 'min_samples_leaf': 10,
 'max_features': 100,
 'max_depth': 10}

In [19]:
rf_classifier = RandomForestClassifier(max_depth=90, max_features=100, min_samples_leaf=10, min_samples_split=80, n_estimators=90)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(max_depth=90, max_features=100, min_samples_leaf=10,
                       min_samples_split=80, n_estimators=90)

search.best_params_: max_depth=10, max_features=8, min_samples_leaf=1, min_samples_split=70, n_estimators=1000

search.best_params_: max_depth=10, max_features=100, min_samples_leaf=10, min_samples_split=80, n_estimators=90

{'n_estimators': 90,
 'min_samples_split': 80,
 'min_samples_leaf': 10,
 'max_features': 100,
 'max_depth': 10}

In [20]:
rf_predictions = rf_classifier.predict(X_test)

In [21]:
mean_absolute_error(y_test, rf_predictions)

0.1831

In [22]:
accuracy_score(y_test, rf_predictions)*100 # %

95.0

In [23]:
precision_score(y_test, rf_predictions, average='macro')*100 # %

94.95352902714428

In [24]:
recall_score(y_test, rf_predictions, average='macro')*100 # %

94.96007576084708

# Saving model


In [27]:
filename = 'final_rf_model3.sav'
pickle.dump(rf_classifier, open(filename, 'wb'))

# Testing with my own digits

In [163]:
img1 = Image.open("data/9.png").convert(mode="1")
array1 = np.array(img1.getdata())

In [164]:
df_myimages = pd.DataFrame(np.array([array1]))

**Predict with my images**

In [165]:
rf_classifier.predict(df_myimages)

array([2], dtype=int64)

# Opening and using saved model

In [30]:
loaded_model = pickle.load(open(filename,'rb'))

In [31]:
loaded_model.predict(df_myimages)

array([5], dtype=int64)